In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.ticker as mtick
import seaborn as sns

# The Data

The data set is comprised of a row per event

- container journeys included are global
    - each journey will contain an update record for every time a change in the shipment data is sourced
    - each update contains all data from prior updates + the new/modified data
    - updates per [reference_id] will not be a 1-for-1 duplicate and will contain at least 1 change. Updates are not generated if no data has changed since the last time data was sourced for a shipment
- multimodal events, e.g. vessel, truck, rail, terminal
- events that have occurred and events estimated to occur are included, see the planned flag
- vessel and location details included
- Many events are reported but the focus will be on the 8 core ocean events:
    - Gate out from origin port
    - Gate in at origin port
    - Loaded on vessel at origin port
    - Vessel departure from origin port
    - Vessel arrived at destination port
    - Discharged from vessel at destination port
    - Gate out from destination port
    - Gate in empty return

In [2]:
df = pd.read_csv('Vizion Senior Data Analyst Code Challenge.csv')

In [3]:
pd.set_option('display.max_columns', None)
df.head(5)

,REFERENCE_ID,REFERENCE_CREATED_AT,REFERENCE_UPDATE_ID,DATA_UPDATED_AT,DESTINATION_PORT,ORIGIN_PORT,RAW_DESCRIPTION,EVENT_DESCRIPTION,EVENT_MODE,EVENT_LOCATION,EVENT_SOURCE,EVENT_TIMESTAMP,PLANNED,CONTAINER_ID,RELATED_VESSEL,CARRIER_NAME
0,07bedccb-b83f-4d87-bc55-8ad54b9755c1,2024-06-19 07:00:14.000 Z,03d9e401-257d-4c47-af37-a4ee52cbe58e,2024-07-15 07:01:01.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Norfolk"",\n ...","{\n ""bic_cd"": null,\n ""city"": ""Chennai"",\n ...",departure,Vessel departure from transshipment port,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Colombo"",\n ...",ais,2024-06-29 07:15:06.000 Z,False,WHLU0639965,"{\n ""imo"": ""9555101"",\n ""mmsi"": ""563172700"",...",Wan Hai
1,09dbaf79-6a76-4efd-a511-663cf9cdd1c2,2024-06-21 22:16:30.000 Z,511e6a19-f4a8-40f0-a7c4-5e1e4d5e90ca,2024-07-30 10:08:45.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Itapoa"",\n ""...","{\n ""bic_cd"": null,\n ""city"": ""Shenzhen"",\n ...",berth,Vessel arrived,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Hong Kong"",\n...",ais,2024-06-18 07:44:20.000 Z,False,PIDU4240363,"{\n ""imo"": ""9793959"",\n ""mmsi"": ""477233200"",...",Pacific International Lines
2,07bedccb-b83f-4d87-bc55-8ad54b9755c1,2024-06-19 07:00:14.000 Z,03d9e401-257d-4c47-af37-a4ee52cbe58e,2024-07-15 07:01:01.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Norfolk"",\n ...","{\n ""bic_cd"": null,\n ""city"": ""Chennai"",\n ...",Inbound/Outbound Laden Ctnr Move From One Pier...,Gate out,Truck,"{\n ""bic_cd"": null,\n ""city"": null,\n ""coun...",carrier,2024-06-29 00:50:00.000 Z,False,WHLU0639965,"{\n ""imo"": null,\n ""mmsi"": null,\n ""vessel""...",Wan Hai
3,924b8c26-7f7a-419b-9d81-c4857a918b8e,2024-06-17 16:50:59.000 Z,2c5eb28d-67fb-4114-9ee3-47cafc7620f7,2024-08-12 16:34:34.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Mobile"",\n ""...","{\n ""bic_cd"": null,\n ""city"": ""Rotterdam"",\n...",departure,Vessel departed,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Veracruz"",\n ...",ais,2024-06-06 11:14:46.000 Z,False,SUDU5331459,"{\n ""imo"": ""9535163"",\n ""mmsi"": ""477346500"",...",Hamburg-Süd
4,001dd4c8-ac21-4042-b762-b6650815b649,2024-06-13 07:00:23.000 Z,bb20cbf7-52b1-48c7-bf10-776dced41dea,2024-07-03 20:19:47.000 Z,"{\n ""city"": ""Savannah"",\n ""country"": ""United...","{\n ""city"": ""Shenzhen"",\n ""country"": ""China""...",Laden Ctnr Loaded On Board Vessel.(Mother Vess...,Loaded on vessel at origin port,Vessel,"{\n ""city"": ""Shenzhen"",\n ""country"": ""China""...",carrier,2024-06-15 01:12:00.000 Z,False,WHLU5809684,"{\n ""imo"": ""9555101"",\n ""mmsi"": ""563172700"",...",Wan Hai


In [4]:
df.shape

(196203, 16)

.

### Task 1:

There are eight core events expected to occur for each journey identified by “reference_id”. What are the rates for how often these events are present for each shipment? You may need to consider whether the container journey is completed for more recent shipments.

- overall
- overall per ocean carrier
- per each of the 8 core events overall
- per each of the 8 core events per ocean carrier

In [5]:
# Overall rate of core events completed by shipment & by ocean carrier

f = pd.DataFrame(((df.groupby(['CARRIER_NAME','REFERENCE_ID'])['EVENT_DESCRIPTION'].nunique()) / (df.groupby(['CARRIER_NAME','REFERENCE_ID'])['REFERENCE_ID'].nunique()*8) *100)
                 .sort_values(ascending=False)).reset_index()

print('Overall completeness rate:',round(f[f[0] >= 100.0]['REFERENCE_ID'].count()/f.REFERENCE_ID.count() * 100,2),'%')
print('***********')
print('***********')
print(f"Overall completeness rate by carrier: \n\
{(round(f[f[0] >= 100.0].groupby('CARRIER_NAME')['REFERENCE_ID'].count()/f.groupby('CARRIER_NAME')['REFERENCE_ID'].count() * 100,2 ))} %")

Overall completeness rate: 91.18 %
***********
***********
Overall completeness rate by carrier: 
CARRIER_NAME
ANL                              100.00
APL                               92.00
CMA-CGM                          100.00
COSCO                             73.91
Hamburg-Süd                      100.00
Hapag Lloyd                      100.00
Hueng A                           84.00
Hyundai Merchant Marine (HMM)    100.00
Korean Marine Transport           96.00
MSC                               95.83
Maersk                            95.00
Matson                            87.50
Namsung Shipping                  96.00
OOCL                              89.47
Ocean Network Express (ONE)      100.00
Pacific International Lines       92.00
RCL Group                        100.00
SM Lines                         100.00
Safmarine                        100.00
Sealand                           91.67
Sinokor                           60.00
Wan Hai                           91.67
Westwood 

In [6]:
# Overall rate for how often all eight of these core events are present for per shipment & per shipment per carrier 

core = ['Gate out from origin port', 'Gate in at origin port', 'Loaded on vessel at origin port', 'Vessel departure from origin port',
        'Vessel arrived at destination port', 'Discharged from vessel at destination port', 'Gate out from destination port', 'Gate in empty return']

cr = df[df.EVENT_DESCRIPTION.isin(core)]

print(f"Overvall rate of completeness per each of the core events: \n\
{round(cr.groupby('EVENT_DESCRIPTION').nunique()['REFERENCE_ID']/cr['REFERENCE_ID'].nunique() * 100,2)}")
print('***********')
print('***********')
print(f"Overvall rate of completeness per each of the core events per ocean carrier \n\
{round(cr.groupby(['CARRIER_NAME','EVENT_DESCRIPTION'])['REFERENCE_ID'].nunique()/cr.groupby('CARRIER_NAME')['REFERENCE_ID'].nunique() * 100,2)}")

Overvall rate of completeness per each of the core events: 
EVENT_DESCRIPTION
Discharged from vessel at destination port    77.30
Gate in at origin port                        83.54
Gate in empty return                          55.81
Gate out from destination port                78.34
Gate out from origin port                     63.78
Loaded on vessel at origin port               86.83
Vessel arrived at destination port            97.05
Vessel departure from origin port             92.55
Name: REFERENCE_ID, dtype: float64
***********
***********
Overvall rate of completeness per each of the core events per ocean carrier 
CARRIER_NAME  EVENT_DESCRIPTION                         
ANL           Discharged from vessel at destination port     91.67
              Gate in at origin port                        100.00
              Gate in empty return                           70.83
              Gate out from destination port                 91.67
              Gate out from origin port      

.

### Task 2:

Import dwell time is calculated as the time from [Discharged from vessel at destination port] to [Gate out from destination port], giving you the time a container dwelled in a terminal before an out gate to the consignee. For shipments that have both of these events can you determine the import dwell time per shipment?EVENT_DESCRIPTION

In [7]:
def dwell_time(df):
    
    
    #ths function calulates and outputs the time acontainer dwelled in a terminal before an out gate to the consignee
    
    # isolate columns for dataframe
    dt = df.copy()
    
    dt = dt[['REFERENCE_ID','CONTAINER_ID','EVENT_DESCRIPTION','EVENT_TIMESTAMP']]
    
    # change event timestamp from object to datetime
    # change datetime format from UTC
    dt['EVENT_DATETIME'] = pd.to_datetime(dt['EVENT_TIMESTAMP'])
    dt['EVENT_DATETIME'] = pd.to_datetime(dt.EVENT_DATETIME).dt.tz_localize(None)
    
    # isolate events and drop duplicated rows
    dt = dt[(dt['EVENT_DESCRIPTION'] == 'Discharged from vessel at destination port') | (dt['EVENT_DESCRIPTION']== 'Gate out from destination port')]
    dt.drop_duplicates(inplace=True)
    
    # group the shipments and isolate which have equal to or less than two unique timestamps
    # this allows us to have the two events per shipment,if applicable 
    timestamps = dt.groupby('REFERENCE_ID').filter(lambda x: len(np.unique(x['EVENT_DATETIME']))<=2).REFERENCE_ID

    # create a new dataframe with the just the shipments with two unique timestamps
    # add timestasmps to new column by grouping the reference ids
    dt2 = dt[dt['REFERENCE_ID'].isin(timestamps)].copy().sort_values(by=['REFERENCE_ID','EVENT_DATETIME'])
    
    for name, group in dt2.groupby('REFERENCE_ID'):
        for i in range(1, len(group)):
            dt2.loc[group.index[0], 'EVENT_DATETIME'+str(i+1)] = group.EVENT_DATETIME.iloc[i]
      
    # rename columns
    dt2.rename(columns={'EVENT_DATETIME2':'OUT','EVENT_DATETIME':'DISCHARGED'},inplace=True)
     
    # caluclate dwell time from when the shipment was discharged to an out gate
    dt2['DWELL_TIME'] = dt2['OUT'] - dt2['DISCHARGED'] 
    
    #drop na to keep one complete row per shipment
    dt2.dropna(how='any',inplace= True)
    
    # dataframe output
    final = pd.DataFrame(dt2.groupby(['REFERENCE_ID','CONTAINER_ID'])['DWELL_TIME'].sum().sort_values(ascending=False)).reset_index()
    
    return final

In [8]:
dw = dwell_time(df)

In [9]:
dw

,REFERENCE_ID,CONTAINER_ID,DWELL_TIME
0,13d52f80-0e70-4218-b56c-0bbd8161e693,CAAU5017730,26 days 14:21:00
1,013ba8c0-133d-4e7a-ab0c-c800249e7088,SEGU3150580,26 days 07:05:00
2,016df906-fc05-47c6-931c-94622fcb2013,WHSU5346522,25 days 02:16:00
3,24f0063b-6fba-42c8-939e-7058a633e14c,MRKU6752945,22 days 07:20:00
4,00c0ce4f-62ab-46b7-8a0f-95d26edb8ad9,SEGU2439795,21 days 19:58:00
...,...,...,...
262,162e885e-48ae-4b91-aad0-a71c2b08eea5,MATU2738597,0 days 01:18:00
263,03a8106f-b0a3-48cd-bf2b-c1562a8dbc75,MATU2627660,0 days 01:17:00
264,2781a302-382b-4b13-9293-bf57b381f5a6,MATU2763898,0 days 00:56:00
265,00332eb9-d3e7-4480-9d72-14397873e155,OOCU4972470,0 days 00:31:00


.

### Task 3:

Are origin and destination port values changing throughout the journey of a container? If so, how many container journeys are experiencing this?

In [10]:
def port_values(x):
    vc = pd.DataFrame(x.groupby('REFERENCE_ID')[['DESTINATION_PORT','ORIGIN_PORT']].nunique()).reset_index()
    print('.......................................................')
    print(vc[(vc['DESTINATION_PORT'] != 1) & (vc['ORIGIN_PORT'] == 1)]['REFERENCE_ID'].count(), 'container journeys had a changed Destination Port value.')
    print(vc[(vc['DESTINATION_PORT'] == 1) & (vc['ORIGIN_PORT'] != 1)]['REFERENCE_ID'].count(),'container journeys had a changed Origin Port value.')
    print(vc[(vc['DESTINATION_PORT'] != 1) & (vc['ORIGIN_PORT'] != 1)]['REFERENCE_ID'].count(),'container journeys had changed Destination Port and Origin Port values.')
    print(vc[(vc['DESTINATION_PORT'] == 1) & (vc['ORIGIN_PORT'] == 1)]['REFERENCE_ID'].count(), 'container journeys had consistent Destination Port and Origin Port values.')
    print('.......................................................')
    return vc


In [11]:
port_values(df)

.......................................................
5 container journeys had a changed Destination Port value.
7 container journeys had a changed Origin Port value.
397 container journeys had changed Destination Port and Origin Port values.
169 container journeys had consistent Destination Port and Origin Port values.
.......................................................


,REFERENCE_ID,DESTINATION_PORT,ORIGIN_PORT
0,000368a8-f6ee-4b64-8703-962cb95d50f7,2,2
1,00037696-362d-4954-b22c-8409a76ebd8b,1,1
2,0003c20e-d825-4be4-b10c-c97ba64a5d14,1,1
3,0003c852-19a1-48c5-bc12-67c02a6b841a,2,2
4,0004450d-97e2-47cd-be1f-bb962adea35a,2,2
...,...,...,...
573,9548a2fb-d218-423c-887a-7f34aa813853,3,2
574,d6af0e06-532b-4feb-83ef-d78a4b95556a,2,2
575,d7a7d294-b01e-475f-929f-ab6e38a2b45e,2,3
576,e6cf9a38-4716-4e4b-9718-485f6a4091b8,1,2


.

### Task 4:

Create a matrix that shows all events reported (not just the 8 core ocean). For these events show:

- which carriers report the event
- if the event has been reported as an estimate, an actual, or both

In [12]:
matrix = df.copy()

In [13]:
matrix['PLANNED_NUM'] = np.where(matrix['PLANNED'] == True,1, 0)

In [14]:
output = matrix.pivot_table(index=['CARRIER_NAME','EVENT_DESCRIPTION'], columns=['PLANNED'],values='PLANNED_NUM',aggfunc="count",fill_value=0)

In [15]:
output

PLANNED                                                False  True
CARRIER_NAME EVENT_DESCRIPTION                                    
ANL          Available for release / delivery              8     0
             Barge arrived                                 0    67
             Barge arrived at destination                  0     4
             Barge departed                                0    92
             Carrier release                               3     0
...                                                      ...   ...
ZIM          Vessel arrived at destination port          134   382
             Vessel arrived at origin port                 2     1
             Vessel departed                             623     0
             Vessel departure from origin port           470    23
             Vessel departure from transshipment port    162   222

[774 rows x 2 columns]

.

In [16]:
#Bonus - - Calculate import dwell time per destination port to determine which ports may perform better in regard to container throughput.
# I extracted (what I think is) the port code, though I would need to work on the spacing & consistency
# Needs further time, just an idea that is being flushed out

import re

In [17]:
df['PORT_CODE'] = df['DESTINATION_PORT'].str.replace('\W', ' ')

/var/folders/pd/q67ljvk140n0pd9zr7_g9k7m0000gn/T/ipykernel_71449/3550658168.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PORT_CODE'] = df['DESTINATION_PORT'].str.replace('\W', ' ')


In [18]:
df['PORT_CODE'].fillna('None',inplace=True)

In [19]:
filtered_df = df[df['PORT_CODE'].str.contains('unlocode', flags=re.IGNORECASE)]

In [20]:
df.loc[filtered_df.index, 'PORT_CODE'] = [re.split('unlocode', row, flags=re.IGNORECASE, maxsplit=1)[-1] for row in filtered_df['PORT_CODE']]

In [21]:
df.head()

,REFERENCE_ID,REFERENCE_CREATED_AT,REFERENCE_UPDATE_ID,DATA_UPDATED_AT,DESTINATION_PORT,ORIGIN_PORT,RAW_DESCRIPTION,EVENT_DESCRIPTION,EVENT_MODE,EVENT_LOCATION,EVENT_SOURCE,EVENT_TIMESTAMP,PLANNED,CONTAINER_ID,RELATED_VESSEL,CARRIER_NAME,PORT_CODE
0,07bedccb-b83f-4d87-bc55-8ad54b9755c1,2024-06-19 07:00:14.000 Z,03d9e401-257d-4c47-af37-a4ee52cbe58e,2024-07-15 07:01:01.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Norfolk"",\n ...","{\n ""bic_cd"": null,\n ""city"": ""Chennai"",\n ...",departure,Vessel departure from transshipment port,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Colombo"",\n ...",ais,2024-06-29 07:15:06.000 Z,False,WHLU0639965,"{\n ""imo"": ""9555101"",\n ""mmsi"": ""563172700"",...",Wan Hai,USORF
1,09dbaf79-6a76-4efd-a511-663cf9cdd1c2,2024-06-21 22:16:30.000 Z,511e6a19-f4a8-40f0-a7c4-5e1e4d5e90ca,2024-07-30 10:08:45.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Itapoa"",\n ""...","{\n ""bic_cd"": null,\n ""city"": ""Shenzhen"",\n ...",berth,Vessel arrived,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Hong Kong"",\n...",ais,2024-06-18 07:44:20.000 Z,False,PIDU4240363,"{\n ""imo"": ""9793959"",\n ""mmsi"": ""477233200"",...",Pacific International Lines,BRIOA
2,07bedccb-b83f-4d87-bc55-8ad54b9755c1,2024-06-19 07:00:14.000 Z,03d9e401-257d-4c47-af37-a4ee52cbe58e,2024-07-15 07:01:01.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Norfolk"",\n ...","{\n ""bic_cd"": null,\n ""city"": ""Chennai"",\n ...",Inbound/Outbound Laden Ctnr Move From One Pier...,Gate out,Truck,"{\n ""bic_cd"": null,\n ""city"": null,\n ""coun...",carrier,2024-06-29 00:50:00.000 Z,False,WHLU0639965,"{\n ""imo"": null,\n ""mmsi"": null,\n ""vessel""...",Wan Hai,USORF
3,924b8c26-7f7a-419b-9d81-c4857a918b8e,2024-06-17 16:50:59.000 Z,2c5eb28d-67fb-4114-9ee3-47cafc7620f7,2024-08-12 16:34:34.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Mobile"",\n ""...","{\n ""bic_cd"": null,\n ""city"": ""Rotterdam"",\n...",departure,Vessel departed,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Veracruz"",\n ...",ais,2024-06-06 11:14:46.000 Z,False,SUDU5331459,"{\n ""imo"": ""9535163"",\n ""mmsi"": ""477346500"",...",Hamburg-Süd,USMOB
4,001dd4c8-ac21-4042-b762-b6650815b649,2024-06-13 07:00:23.000 Z,bb20cbf7-52b1-48c7-bf10-776dced41dea,2024-07-03 20:19:47.000 Z,"{\n ""city"": ""Savannah"",\n ""country"": ""United...","{\n ""city"": ""Shenzhen"",\n ""country"": ""China""...",Laden Ctnr Loaded On Board Vessel.(Mother Vess...,Loaded on vessel at origin port,Vessel,"{\n ""city"": ""Shenzhen"",\n ""country"": ""China""...",carrier,2024-06-15 01:12:00.000 Z,False,WHLU5809684,"{\n ""imo"": ""9555101"",\n ""mmsi"": ""563172700"",...",Wan Hai,USSAV


In [22]:
k = df[['REFERENCE_ID','CONTAINER_ID','PORT_CODE']]

In [23]:
k = k.drop_duplicates()

In [24]:
k

,REFERENCE_ID,CONTAINER_ID,PORT_CODE
0,07bedccb-b83f-4d87-bc55-8ad54b9755c1,WHLU0639965,USORF
1,09dbaf79-6a76-4efd-a511-663cf9cdd1c2,PIDU4240363,BRIOA
3,924b8c26-7f7a-419b-9d81-c4857a918b8e,SUDU5331459,USMOB
4,001dd4c8-ac21-4042-b762-b6650815b649,WHLU5809684,USSAV
5,040eab22-de1b-496f-8ef4-91c98c6d482b,CGMU3065137,AUMEL
...,...,...,...
187454,00329dd4-1c7d-40cd-a01d-6a59b1590db9,MSMU5102357,None
189841,00a2b342-b410-4425-b589-869cb605132a,TRHU6866780,USNYC
192417,0301a798-b833-4884-b642-da00f5a54e9c,TRHU4620230,AUFRE
193560,00549938-d3f4-4342-b92d-2ac36512c9dc,TTNU8153112,FRUS2


In [25]:
dw

,REFERENCE_ID,CONTAINER_ID,DWELL_TIME
0,13d52f80-0e70-4218-b56c-0bbd8161e693,CAAU5017730,26 days 14:21:00
1,013ba8c0-133d-4e7a-ab0c-c800249e7088,SEGU3150580,26 days 07:05:00
2,016df906-fc05-47c6-931c-94622fcb2013,WHSU5346522,25 days 02:16:00
3,24f0063b-6fba-42c8-939e-7058a633e14c,MRKU6752945,22 days 07:20:00
4,00c0ce4f-62ab-46b7-8a0f-95d26edb8ad9,SEGU2439795,21 days 19:58:00
...,...,...,...
262,162e885e-48ae-4b91-aad0-a71c2b08eea5,MATU2738597,0 days 01:18:00
263,03a8106f-b0a3-48cd-bf2b-c1562a8dbc75,MATU2627660,0 days 01:17:00
264,2781a302-382b-4b13-9293-bf57b381f5a6,MATU2763898,0 days 00:56:00
265,00332eb9-d3e7-4480-9d72-14397873e155,OOCU4972470,0 days 00:31:00


In [26]:
test = pd.merge(k, dw, on=['REFERENCE_ID','CONTAINER_ID'], how="left")

In [27]:
test

,REFERENCE_ID,CONTAINER_ID,PORT_CODE,DWELL_TIME
0,07bedccb-b83f-4d87-bc55-8ad54b9755c1,WHLU0639965,USORF,NaT
1,09dbaf79-6a76-4efd-a511-663cf9cdd1c2,PIDU4240363,BRIOA,NaT
2,924b8c26-7f7a-419b-9d81-c4857a918b8e,SUDU5331459,USMOB,8 days 05:31:00
3,001dd4c8-ac21-4042-b762-b6650815b649,WHLU5809684,USSAV,NaT
4,040eab22-de1b-496f-8ef4-91c98c6d482b,CGMU3065137,AUMEL,2 days 08:28:00
...,...,...,...,...
646,00329dd4-1c7d-40cd-a01d-6a59b1590db9,MSMU5102357,None,1 days 00:00:00
647,00a2b342-b410-4425-b589-869cb605132a,TRHU6866780,USNYC,2 days 16:56:00
648,0301a798-b833-4884-b642-da00f5a54e9c,TRHU4620230,AUFRE,0 days 05:07:00
649,00549938-d3f4-4342-b92d-2ac36512c9dc,TTNU8153112,FRUS2,1 days 00:00:00


In [28]:
test.dropna(how='any',inplace=True)

In [29]:
test = test.drop_duplicates()

In [30]:
test

,REFERENCE_ID,CONTAINER_ID,PORT_CODE,DWELL_TIME
2,924b8c26-7f7a-419b-9d81-c4857a918b8e,SUDU5331459,USMOB,8 days 05:31:00
4,040eab22-de1b-496f-8ef4-91c98c6d482b,CGMU3065137,AUMEL,2 days 08:28:00
6,00329dd4-1c7d-40cd-a01d-6a59b1590db9,MSMU5102357,DEHAM,1 days 00:00:00
8,0e53d881-57a0-4c34-bc85-4212abf376ed,MRKU0511914,CNSGH,2 days 12:01:00
9,0ad78139-22f2-48ca-829b-1f39ddfebc52,CAAU8086777,AUMEL,2 days 18:26:00
...,...,...,...,...
645,016df906-fc05-47c6-931c-94622fcb2013,WHSU5346522,MYPEN,25 days 02:16:00
646,00329dd4-1c7d-40cd-a01d-6a59b1590db9,MSMU5102357,None,1 days 00:00:00
647,00a2b342-b410-4425-b589-869cb605132a,TRHU6866780,USNYC,2 days 16:56:00
648,0301a798-b833-4884-b642-da00f5a54e9c,TRHU4620230,AUFRE,0 days 05:07:00


In [31]:
print('The most efficient ports with the shortest total dwell time per shipment\:',test.groupby(['CONTAINER_ID','PORT_CODE'])['DWELL_TIME'].sum().sort_values().nsmallest(5))

The most efficient ports with the shortest total dwell time per shipment\: CONTAINER_ID  PORT_CODE   
MATU2763563       USLGB      0 days 00:25:00
OOCU4972470       USNYC      0 days 00:31:00
MATU2763898       USLGB      0 days 00:56:00
MATU2627660       USLGB      0 days 01:17:00
MATU2738597       USLGB      0 days 01:18:00
Name: DWELL_TIME, dtype: timedelta64[ns]
